In [9]:
%load_ext autoreload
%autoreload
import tensorflow as tf
import numpy as np
import surreal_single, config
import functools, operator, copy
import tensorflow.contrib.slim as slim
from nets import resnet_v2
from nets.mobilenet import mobilenet_v2
from tensor_info2 import INPUT_TENSOR_INFO, OUTPUT_TENSOR_INFO
tf.reset_default_graph()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
types = tuple(t['type'] for t in INPUT_TENSOR_INFO)
config.BATCH_SIZE = 10
input_tensors = tf.data.Dataset.from_generator(surreal_single.load, types) \
                               .batch(config.BATCH_SIZE) \
                               .prefetch(config.PREFETCH_SIZE) \
                               .make_one_shot_iterator() \
                               .get_next()
tensors = {}
for tensor, info in zip(input_tensors, INPUT_TENSOR_INFO):
    tensor.set_shape(info['shape'])
    tensors[info['name']] = tensor
print(tensors)

{'image': <tf.Tensor 'IteratorGetNext:0' shape=(10, 401, 401, 3) dtype=float32>, 'hm': <tf.Tensor 'IteratorGetNext:1' shape=(10, 401, 401, 16) dtype=float32>, 'so_x': <tf.Tensor 'IteratorGetNext:2' shape=(10, 401, 401, 16) dtype=float32>, 'so_y': <tf.Tensor 'IteratorGetNext:3' shape=(10, 401, 401, 16) dtype=float32>, 'mo_x': <tf.Tensor 'IteratorGetNext:4' shape=(10, 401, 401, 30) dtype=float32>, 'mo_y': <tf.Tensor 'IteratorGetNext:5' shape=(10, 401, 401, 30) dtype=float32>}


In [11]:
config.STRIDE = 16

MD_H = int((config.TAR_H-1)//config.STRIDE)+1
MD_W = int((config.TAR_W-1)//config.STRIDE)+1

DEPTH = [ti['shape'][-1] for ti in OUTPUT_TENSOR_INFO]
RESULT_SHAPE = (config.BATCH_SIZE, MD_H, MD_W, sum(DEPTH))
RESULT_SIZE = functools.reduce(operator.mul, RESULT_SHAPE[1:])
OUTPUT_SHAPE = (config.BATCH_SIZE, config.TAR_H, config.TAR_W, sum(DEPTH))
OUTPUT_SIZE = functools.reduce(operator.mul, OUTPUT_SHAPE[1:])


In [12]:
def bilinear(indices):
    oy = tf.clip_by_value(indices[1], 0, MD_H-1)
    ox = tf.clip_by_value(indices[2], 0, MD_W-1)
    iy = [tf.floor(oy), tf.ceil(oy + 1e-9)]
    ix = [tf.floor(ox), tf.ceil(ox + 1e-9)]
    idx_p = []
    for y in iy:
        for x in ix:
            indices[1] = y
            indices[2] = x
            idx = tf.cast(tf.stack(indices, axis=-1), tf.int32)
            p = (1 - tf.abs(y - oy)) * (1 - tf.abs(x - ox))
            idx_p.append((idx, p))
    return idx_p

def gather_bilinear(params, indices):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.gather_nd(params, idx)
        res.append(r * p)
    return tf.add_n(res)

def scatter_bilinear(params, indices, shape):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.scatter_nd(idx, params, shape)
        if len(r.shape) > len(p.shape):
            p = tf.expand_dims(p, axis=-1)
        res.append(r * p)
    return tf.add_n(res)

In [13]:
def resize(tensor):
    return tf.image.resize_images(
        tensor,
        (config.TAR_H, config.TAR_W),
        method=tf.image.ResizeMethod.BILINEAR,
        align_corners=True)

with slim.arg_scope(resnet_v2.resnet_arg_scope()):
    _, end_points = resnet_v2.resnet_v2_101(tensors['image'], output_stride=config.STRIDE)
    model_output = end_points['resnet_v2_101/block4']
    print('model_output', model_output)
    hm_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1], activation_fn=tf.sigmoid)
    so_x_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1], activation_fn=None)
    so_y_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1], activation_fn=None)
    mo_x_pred = slim.conv2d(model_output, config.NUM_EDGE, [1, 1], activation_fn=None)
    mo_y_pred = slim.conv2d(model_output, config.NUM_EDGE, [1, 1], activation_fn=None)

    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)
    b, y, x, i = np.mgrid[:config.BATCH_SIZE, :MD_H, :MD_W, :config.NUM_EDGE]
    for _ in range(config.NUM_RECURRENT):
        mo_p = [b, y+mo_y_pred, x+mo_x_pred, i]
        mo_x_pred = gather_bilinear(so_x_pred, mo_p) + mo_x_pred
        mo_y_pred = gather_bilinear(so_y_pred, mo_p) + mo_y_pred
    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)
    hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred = [resize(x) for x in [hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred]]
    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)
    so_x_pred, so_y_pred, mo_x_pred, mo_y_pred = [x * config.STRIDE for x in [so_x_pred, so_y_pred, mo_x_pred, mo_y_pred]]
    print(hm_pred, so_x_pred, so_y_pred, mo_x_pred, mo_y_pred)

model_output Tensor("resnet_v2_101/block4/unit_3/bottleneck_v2/add:0", shape=(10, 26, 26, 2048), dtype=float32)
Tensor("Conv/Sigmoid:0", shape=(10, 26, 26, 16), dtype=float32) Tensor("Conv_1/BatchNorm/FusedBatchNorm:0", shape=(10, 26, 26, 16), dtype=float32) Tensor("Conv_2/BatchNorm/FusedBatchNorm:0", shape=(10, 26, 26, 16), dtype=float32) Tensor("Conv_3/BatchNorm/FusedBatchNorm:0", shape=(10, 26, 26, 30), dtype=float32) Tensor("Conv_4/BatchNorm/FusedBatchNorm:0", shape=(10, 26, 26, 30), dtype=float32)
Tensor("Conv/Sigmoid:0", shape=(10, 26, 26, 16), dtype=float32) Tensor("Conv_1/BatchNorm/FusedBatchNorm:0", shape=(10, 26, 26, 16), dtype=float32) Tensor("Conv_2/BatchNorm/FusedBatchNorm:0", shape=(10, 26, 26, 16), dtype=float32) Tensor("add_12:0", shape=(10, 26, 26, 30), dtype=float32) Tensor("add_15:0", shape=(10, 26, 26, 30), dtype=float32)
Tensor("resize_images/ResizeBilinear:0", shape=(10, 401, 401, 16), dtype=float32) Tensor("resize_images_1/ResizeBilinear:0", shape=(10, 401, 401, 

In [14]:
hm_loss = - tf.reduce_mean(tensors['hm'] * tf.log(hm_pred + 1e-9) + (1 - tensors['hm']) * tf.log(1 - hm_pred + 1e-9))
so_loss = tf.abs(tensors['so_x'] - so_x_pred) / config.RADIUS + tf.abs(tensors['so_y'] - so_y_pred) / config.RADIUS
mo_loss = tf.abs(tensors['mo_x'] - mo_x_pred) / config.RADIUS + tf.abs(tensors['mo_y'] - mo_y_pred) / config.RADIUS

disc_only = tf.cast(tensors['hm'], tf.float32)
disc_size = tf.reduce_sum(disc_only, axis=[1, 2]) + 1e-9
so_loss = tf.reduce_mean(tf.reduce_sum(so_loss * disc_only, axis=[1, 2]) / disc_size)

disc_only = tf.cast(tf.gather(tensors['hm'], config.EDGES[:, 0], axis=-1), tf.float32)
disc_size = tf.reduce_sum(disc_only, axis=[1, 2]) + 1e-9
mo_loss = tf.reduce_mean(tf.reduce_sum(mo_loss * disc_only, axis=[1, 2]) / disc_size)

In [15]:
total_loss = hm_loss * 4.0 + so_loss * 1.0 + mo_loss * 0.5

In [16]:
tf.summary.scalar('losses/hm_loss', hm_loss)
tf.summary.scalar('losses/so_loss', so_loss)
tf.summary.scalar('losses/mo_loss', mo_loss)
tf.summary.scalar('losses/total_loss', total_loss)

<tf.Tensor 'losses/total_loss:0' shape=() dtype=string>

In [17]:
tf.summary.histogram("pred_dist/resnet", model_output)
tf.summary.histogram("pred_dist/heatmap", hm_pred)
tf.summary.histogram("pred_dist/short_off_x", so_x_pred)
tf.summary.histogram("pred_dist/short_off_y", so_y_pred)
tf.summary.histogram("pred_dist/mid_off_x", mo_x_pred)
tf.summary.histogram("pred_dist/mid_off_y", mo_y_pred)

tf.summary.histogram("true_dist/heatmap", tensors['hm'])
tf.summary.histogram("true_dist/short_off_x", tensors['so_x'])
tf.summary.histogram("true_dist/short_off_y", tensors['so_y'])
tf.summary.histogram("true_dist/mid_off_x", tensors['mo_x'])
tf.summary.histogram("true_dist/mid_off_y", tensors['mo_y'])

optimizer = tf.train.AdamOptimizer()
train_op = slim.learning.create_train_op(total_loss, optimizer)

checkpoint_path = 'resnet/resnet_v2_101.ckpt'
variables = slim.get_model_variables()
restore_map = {}
for v in variables:
    if not v.name.startswith('resnet'):
        continue
    org_name = v.name.split(':')[0]
    restore_map[org_name] = v
    print(org_name, ':', v.name)
init_assign_op, init_feed_dict = slim.assign_from_checkpoint(checkpoint_path, restore_map)

resnet_v2_101/conv1/weights : resnet_v2_101/conv1/weights:0
resnet_v2_101/conv1/biases : resnet_v2_101/conv1/biases:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0
resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance : resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0
resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights : resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0
resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/biases : resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/biases:0
resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/weights : resnet_v2_101/block1/unit_1/bottleneck_v2/conv1/weights:0
resnet_v2_101/block1/unit_1/bottleneck

In [18]:
import time, os, shutil
#log_dir = 'logs/log_' + str(time.time())[-5:]
log_dir = 'logs/res_log_test'
shutil.rmtree(log_dir)
os.mkdir(log_dir)

In [ ]:
def InitAssignFn(sess):
    sess.run(init_assign_op, init_feed_dict)
tf.contrib.slim.learning.train(train_op,
                               '/home/ubuntu/personlab/'+log_dir,
                               init_fn=InitAssignFn,
                               log_every_n_steps=100,
                               save_summaries_secs=30,
                              )

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global_step/sec: 0.350406
INFO:tensorflow:Recording summary at step 7.
INFO:tensorflow:global_step/sec: 0.398733
INFO:tensorflow:Recording summary at step 20.
INFO:tensorflow:global_step/sec: 0.437153
INFO:tensorflow:Recording summary at step 32.
INFO:tensorflow:global_step/sec: 0.434854
INFO:tensorflow:Recording summary at step 45.
INFO:tensorflow:global_step/sec: 0.434658
INFO:tensorflow:Recording summary at step 59.
INFO:tensorflow:global_step/sec: 0.360985
INFO:tensorflow:Recording summary at step 70.
INFO:tensorflow:global_step/sec: 0.398306
INFO:tensorflow:Recording sum

INFO:tensorflow:Recording summary at step 1496.
INFO:tensorflow:global step 1499: loss = 1.7111 (1.886 sec/step)
INFO:tensorflow:Recording summary at step 1508.
INFO:tensorflow:Recording summary at step 1520.
INFO:tensorflow:Recording summary at step 1534.
INFO:tensorflow:Recording summary at step 1544.
INFO:tensorflow:Recording summary at step 1556.
INFO:tensorflow:Recording summary at step 1568.
INFO:tensorflow:Recording summary at step 1580.
INFO:tensorflow:Recording summary at step 1593.
INFO:tensorflow:global step 1599: loss = 1.5196 (1.879 sec/step)
INFO:tensorflow:Recording summary at step 1604.
INFO:tensorflow:Recording summary at step 1617.
INFO:tensorflow:Recording summary at step 1629.
INFO:tensorflow:Recording summary at step 1640.
INFO:tensorflow:Recording summary at step 1653.
INFO:tensorflow:Recording summary at step 1665.
INFO:tensorflow:Recording summary at step 1677.
INFO:tensorflow:Recording summary at step 1689.
INFO:tensorflow:global step 1699: loss = 1.8080 (1.898

INFO:tensorflow:Recording summary at step 3166.
INFO:tensorflow:Recording summary at step 3178.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Recording summary at step 3191.
INFO:tensorflow:global step 3199: loss = 1.0738 (1.872 sec/step)
INFO:tensorflow:Recording summary at step 3202.
INFO:tensorflow:Recording summary at step 3214.
INFO:tensorflow:Recording summary at step 3227.
INFO:tensorflow:Recording summary at step 3238.
INFO:tensorflow:Recording summary at step 3251.
INFO:tensorflow:Recording summary at step 3264.
INFO:tensorflow:Recording summary at step 3275.
INFO:tensorflow:Recording summary at step 3288.
INFO:tensorflow:global step 3299: loss = 1.4262 (1.870 sec/step)
INFO:tensorflow:Recording summary at step 3300.
INFO:tensorflow:Recording summary at step 3313.
INFO:tensorflow:Recording summary at step 3326.
INFO:tensorflow:Recording summary at step 3337.
INFO:tensorflow:Recording summary at step 3350.
INFO:ten

INFO:tensorflow:Recording summary at step 4815.
INFO:tensorflow:Recording summary at step 4826.
INFO:tensorflow:Recording summary at step 4838.
INFO:tensorflow:Recording summary at step 4852.
INFO:tensorflow:Recording summary at step 4863.
INFO:tensorflow:Recording summary at step 4876.
INFO:tensorflow:Recording summary at step 4889.
INFO:tensorflow:global step 4899: loss = 1.0215 (1.868 sec/step)
INFO:tensorflow:Recording summary at step 4900.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Recording summary at step 4912.
INFO:tensorflow:Recording summary at step 4925.
INFO:tensorflow:Recording summary at step 4936.
INFO:tensorflow:Recording summary at step 4949.
INFO:tensorflow:Recording summary at step 4962.
INFO:tensorflow:Recording summary at step 4972.
INFO:tensorflow:Recording summary at step 4985.
INFO:tensorflow:Recording summary at step 4998.
INFO:tensorflow:global step 4999: loss = 0.9626 (5.676 sec/step)
INFO:ten

INFO:tensorflow:Recording summary at step 6493.
INFO:tensorflow:global step 6499: loss = 0.7753 (1.896 sec/step)
INFO:tensorflow:Recording summary at step 6506.
INFO:tensorflow:Recording summary at step 6517.
INFO:tensorflow:Recording summary at step 6530.
INFO:tensorflow:Recording summary at step 6542.
INFO:tensorflow:Recording summary at step 6554.
INFO:tensorflow:Recording summary at step 6567.
INFO:tensorflow:Recording summary at step 6579.
INFO:tensorflow:Recording summary at step 6591.
INFO:tensorflow:global step 6599: loss = 0.7172 (1.882 sec/step)
INFO:tensorflow:Recording summary at step 6604.
INFO:tensorflow:Recording summary at step 6615.
INFO:tensorflow:Recording summary at step 6628.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Recording summary at step 6641.
INFO:tensorflow:Recording summary at step 6652.
INFO:tensorflow:Recording summary at step 6666.
INFO:tensorflow:Recording summary at step 6679.
INFO:ten

...

INFO:tensorflow:Recording summary at step 27837.
INFO:tensorflow:Recording summary at step 27848.
INFO:tensorflow:Recording summary at step 27861.
INFO:tensorflow:Recording summary at step 27874.
INFO:tensorflow:Recording summary at step 27886.
INFO:tensorflow:global step 27899: loss = 0.6201 (2.378 sec/step)
INFO:tensorflow:Recording summary at step 27898.
INFO:tensorflow:Recording summary at step 27910.
INFO:tensorflow:Recording summary at step 27922.
INFO:tensorflow:Recording summary at step 27935.
INFO:tensorflow:Recording summary at step 27948.
INFO:tensorflow:Recording summary at step 27960.
INFO:tensorflow:Recording summary at step 27973.
INFO:tensorflow:Recording summary at step 27985.
INFO:tensorflow:Recording summary at step 27997.
INFO:tensorflow:global step 27999: loss = 0.5697 (1.888 sec/step)
INFO:tensorflow:Recording summary at step 28009.
INFO:tensorflow:Recording summary at step 28021.
INFO:tensorflow:Recording summary at step 28034.
INFO:tensorflow:Recording summary a

INFO:tensorflow:global step 29499: loss = 0.4758 (1.900 sec/step)
INFO:tensorflow:Recording summary at step 29501.
INFO:tensorflow:Recording summary at step 29515.
INFO:tensorflow:Recording summary at step 29526.
INFO:tensorflow:Recording summary at step 29539.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Recording summary at step 29552.
INFO:tensorflow:Recording summary at step 29563.
INFO:tensorflow:Recording summary at step 29576.
INFO:tensorflow:Recording summary at step 29588.
INFO:tensorflow:global step 29599: loss = 0.4287 (1.863 sec/step)
INFO:tensorflow:Recording summary at step 29600.
INFO:tensorflow:Recording summary at step 29613.
INFO:tensorflow:Recording summary at step 29626.
INFO:tensorflow:Recording summary at step 29637.
INFO:tensorflow:Recording summary at step 29650.
INFO:tensorflow:Recording summary at step 29663.
INFO:tensorflow:Recording summary at step 29674.
INFO:tensorflow:Recording summary at st

INFO:tensorflow:Recording summary at step 31120.
INFO:tensorflow:Recording summary at step 31132.
INFO:tensorflow:Recording summary at step 31145.
INFO:tensorflow:Recording summary at step 31157.
INFO:tensorflow:Recording summary at step 31169.
INFO:tensorflow:Recording summary at step 31182.
INFO:tensorflow:Recording summary at step 31194.
INFO:tensorflow:global step 31199: loss = 0.7329 (1.866 sec/step)
INFO:tensorflow:Recording summary at step 31206.
INFO:tensorflow:Recording summary at step 31219.
INFO:tensorflow:Recording summary at step 31231.
INFO:tensorflow:Recording summary at step 31244.
INFO:tensorflow:Recording summary at step 31257.
INFO:tensorflow:Recording summary at step 31268.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Recording summary at step 31281.
INFO:tensorflow:Recording summary at step 31293.
INFO:tensorflow:global step 31299: loss = 0.7998 (1.889 sec/step)
INFO:tensorflow:Recording summary at st

INFO:tensorflow:Recording summary at step 32757.
INFO:tensorflow:Recording summary at step 32769.
INFO:tensorflow:Recording summary at step 32782.
INFO:tensorflow:Recording summary at step 32795.
INFO:tensorflow:global step 32799: loss = 0.3740 (1.872 sec/step)
INFO:tensorflow:Recording summary at step 32807.
INFO:tensorflow:Recording summary at step 32819.
INFO:tensorflow:Recording summary at step 32832.
INFO:tensorflow:Recording summary at step 32844.
INFO:tensorflow:Recording summary at step 32857.
INFO:tensorflow:Recording summary at step 32869.
INFO:tensorflow:Recording summary at step 32881.
INFO:tensorflow:Recording summary at step 32894.
INFO:tensorflow:global step 32899: loss = 0.7801 (1.876 sec/step)
INFO:tensorflow:Recording summary at step 32907.
INFO:tensorflow:Recording summary at step 32919.
INFO:tensorflow:Recording summary at step 32932.
INFO:tensorflow:Recording summary at step 32944.
INFO:tensorflow:Recording summary at step 32956.
INFO:tensorflow:Recording summary a

INFO:tensorflow:Recording summary at step 34409.
INFO:tensorflow:Recording summary at step 34420.
INFO:tensorflow:Recording summary at step 34432.
INFO:tensorflow:Recording summary at step 34445.
INFO:tensorflow:Recording summary at step 34457.
INFO:tensorflow:Recording summary at step 34470.
INFO:tensorflow:Saving checkpoint to path /home/ubuntu/personlab/logs/res_log_test/model.ckpt
INFO:tensorflow:Recording summary at step 34482.
INFO:tensorflow:Recording summary at step 34493.
INFO:tensorflow:global step 34499: loss = 0.3646 (1.885 sec/step)
INFO:tensorflow:Recording summary at step 34507.
INFO:tensorflow:Recording summary at step 34519.
INFO:tensorflow:Recording summary at step 34531.
INFO:tensorflow:Recording summary at step 34544.
INFO:tensorflow:Recording summary at step 34557.
INFO:tensorflow:Recording summary at step 34568.
INFO:tensorflow:Recording summary at step 34581.
INFO:tensorflow:Recording summary at step 34592.
INFO:tensorflow:global step 34599: loss = 0.5620 (1.903 

In [ ]:
from tensorflow.python.tools import inspect_checkpoint
inspect_checkpoint.print_tensors_in_checkpoint_file(checkpoint_path, tensor_name='', all_tensors=False, all_tensor_names=True)

In [ ]:

saver = tf.train.Saver()
with tf.device('/cpu:0'):
    with tf.Session() as sess:
        checkpoint_path = 'logs/res_log_test/model.ckpt-4575'
        saver.restore(sess, checkpoint_path)
        m_out, img, hm = sess.run([model_output, tensors['image'], hm_out])


In [ ]:
import cv2
from matplotlib import pyplot as plt

def overlay(img, over, alpha=0.5):
    out = img.copy()
    if img.max() > 1.:
        out = out / 255.
    out *= 1-alpha
    if len(over.shape)==2:
        out += alpha*over[:,:,np.newaxis]
    else:
        out += alpha*over    
    return out
b_i = 0
plt.figure()
plt.imshow(overlay(img[b_i, ...], np.max(hm[b_i, ...], axis=-1), alpha=0.7))

In [ ]:
m_out

In [ ]:
config.EDGES[:, ::-1]